1. Local development 
2. move into GPU EC2 node
3. Add DDP

In [1]:
# TODO: activate venv somehow
# enable importing python files without caching them
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install -r requirements.txt

In [13]:
# 1. prepare dataset
from src.download_multinode_dataset import download_dataset

dataset_path = '../temp/train_datasets/mnist'
download_dataset(dataset_path)

ahoj


In [19]:

# TODO: create temp if not exists???
# throws errors:
# W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
# [W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
# [W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).

# 2. run Trainer class via torchrun on my localhost machine
!torchrun --standalone \
    ./src/main_mnist_multinode.py \
    --total_epochs 5 \
    --save_every 5 \
    --experiment_name xxxx \
    --dataset_path {dataset_path} \
    --snapshots_dir ../temp/snapshots

NOTE: Redirects are currently not supported in Windows or MacOs.
master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
[GR:0LR:0] is_main_log_node True
[GR:0LR:0] device: cpu
[GR:0LR:0] <function get_trainset at 0x14243cea0>
[W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addr